# Install Packages and Setup Variables


In [1]:
!pip install -q llama-index==0.12.37 openai==1.59.8 llama-index-finetuning==0.3.1 llama-index-embeddings-huggingface==0.5.1 llama-index-embeddings-cohere==0.4.0 \
                llama-index-readers-web==0.3.5 cohere==5.15.0 tiktoken==0.8.0 chromadb==1.0.11 html2text==2024.2.26 sentence-transformers==4.1.0 pydantic==2.11.0 \
                llama-index-vector-stores-chroma==0.4.2 kaleido==0.2.1 llama-index-llms-gemini==0.4.1 typing_extensions==4.14.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 90.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.6/455.6 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 6.

In [2]:
import os

# Set the following API Keys in the Python environment. Will be used later.
os.environ["OPENAI_API_KEY"] = "<YOUR-OPENAI-API-KEY>"
os.environ["GOOGLE_API_KEY"] = "<YOUR-GOOGLE-API-KEY>"

# from google.colab import userdata
# os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key')
# os.environ["GOOGLE_API_KEY"] = userdata.get('Google_api_key')

In [3]:
# Allows running asyncio in environments with an existing event loop, like Jupyter notebooks.
import nest_asyncio

nest_asyncio.apply()

# Load a Model


In [4]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

Settings.llm = OpenAI(temperature=0, model="gpt-4o-mini")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

# Load Indexes


In [5]:
from huggingface_hub import hf_hub_download

vectorstore = hf_hub_download(repo_id="jaiganesan/ai_tutor_knowledge", filename="vectorstore.zip", repo_type="dataset", local_dir=".")

vectorstore.zip:   0%|          | 0.00/97.2M [00:00<?, ?B/s]

In [6]:
!unzip -o vectorstore.zip

Archive:  vectorstore.zip
   creating: ai_tutor_knowledge/
   creating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/length.bin  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/index_metadata.pickle  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/link_lists.bin  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/header.bin  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/data_level0.bin  
  inflating: ai_tutor_knowledge/chroma.sqlite3  


In [7]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import VectorStoreIndex

# Create the vector index
db = chromadb.PersistentClient(path="./ai_tutor_knowledge")
chroma_collection = db.get_or_create_collection("ai_tutor_knowledge")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
vector_index = VectorStoreIndex.from_vector_store(vector_store)

# Multi-Step Query Engine


## GPT-4o-mini


In [8]:
from llama_index.core.indices.query.query_transform.base import (
    StepDecomposeQueryTransform,
)

step_decompose_transform_gpt4o = StepDecomposeQueryTransform(verbose=True, llm=Settings.llm)

In [9]:
from llama_index.core.query_engine.multistep_query_engine import MultiStepQueryEngine

#Default query engine
query_engine_gpt4o_mini = vector_index.as_query_engine()

# Multi Step Query Engine
multi_step_query_engine = MultiStepQueryEngine(
    query_engine = query_engine_gpt4o_mini,
    query_transform = step_decompose_transform_gpt4o,
    index_summary = "Used to answer the Questions about RAG, Machine Learning, Deep Learning, and Generative AI, Note: Don't repeat the Same quesion",
)

# Query Dataset

## Default

In [10]:
# Default query engine
query_engine = vector_index.as_query_engine()
res = query_engine.query("Write about Llama 3.1 Model, BERT and PEFT methods")
print(res.response)

The LLaMA model is a foundational model designed for various natural language processing tasks. It can be fine-tuned using the Parameter-Efficient Fine-Tuning (PEFT) library, which includes methods like LoRA (Low-Rank Adaptation) and Llama-Adapter. LoRA allows for efficient fine-tuning by introducing a small number of learnable parameters, making it suitable for scenarios with limited computational resources. Llama-Adapter specifically transforms the LLaMA model into an instruction-following model by integrating learnable adaptation prompts while preserving the model's pre-trained knowledge.

BERT, another prominent model in the NLP landscape, is known for its bidirectional training approach, which allows it to understand context from both directions in a sentence. While BERT is not directly mentioned in the provided context, it is often used in conjunction with fine-tuning techniques similar to those offered by the PEFT library.

Overall, the PEFT methods provide efficient ways to ada

In [11]:
for src in res.source_nodes:
    print("Node ID\t", src.node_id)
    print("Title\t", src.metadata["title"])
    print("Text\t", src.text)
    print("Score\t", src.score)
    print("-_" * 20)

Node ID	 781b7b12-eca2-47c0-a66e-9d6be670e951
Title	 LLaMA
Text	 on how to fine-tune LLaMA model using LoRA method via the 🤗 PEFT library with intuitive UI. 🌎 - A [notebook](https://github.com/aws/amazon-sagemaker-examples/blob/main/introduction_to_amazon_algorithms/jumpstart-foundation-models/text-generation-open-llama.ipynb) on how to deploy Open-LLaMA model for text generation on Amazon SageMaker. 🌎 ## LlamaConfig[[autodoc]] LlamaConfig## LlamaTokenizer[[autodoc]] LlamaTokenizer    - build_inputs_with_special_tokens    - get_special_tokens_mask    - create_token_type_ids_from_sequences    - save_vocabulary## LlamaTokenizerFast[[autodoc]] LlamaTokenizerFast    - build_inputs_with_special_tokens    - get_special_tokens_mask    - create_token_type_ids_from_sequences    - update_post_processor    - save_vocabulary## LlamaModel[[autodoc]] LlamaModel    - forward## LlamaForCausalLM[[autodoc]] LlamaForCausalLM    - forward## LlamaForSequenceClassification[[autodoc]] LlamaForSequenceClassif

## GPT-4o-mini Multi-Step


In [12]:
response = multi_step_query_engine.query("Write about Llama 3.1 Model, BERT and PEFT methods")
print(response.response)

> Current query: Write about Llama 3.1 Model, BERT and PEFT methods
> New query: What are the key features of the Llama 3.1 Model?
> Current query: Write about Llama 3.1 Model, BERT and PEFT methods
> New query: What are the key features of the Llama 3.1 Model?
> Current query: Write about Llama 3.1 Model, BERT and PEFT methods
> New query: What are the key features of BERT?
The Llama 3.1 model is a state-of-the-art AI language model known for its impressive scale and capabilities. It is the largest model developed by Meta, trained on over 15 trillion tokens using more than 16,000 H100 GPUs. One of its standout features is the extended context length of 128K, which allows it to process longer inputs effectively. The model exhibits enhanced reasoning and coding abilities, supports zero-shot tool use, and has a strong multilingual processing capability, with about 50% of its training data being multilingual. Additionally, Llama 3.1 excels in programming and reasoning skills, generating h

In [13]:
for query, response in response.metadata['sub_qa']:
    print(f"**{query}**\n{response}\n")

**What are the key features of the Llama 3.1 Model?**
The Llama 3.1 model boasts several key features, including:

1. **Model Scale and Training**: It is the largest model from Meta, trained on over 15 trillion tokens using more than 16,000 H100 GPUs.

2. **Extended Context Length**: The model supports a context length of 128K, enhancing its ability to handle longer inputs.

3. **Enhanced Capabilities**: It demonstrates improved reasoning and coding abilities compared to its predecessors.

4. **Tool Use and Agentic Behaviors**: The model supports zero-shot tool use, allowing it to perform tasks without prior training on specific tools.

5. **Multilingual Processing**: Approximately 50% of its training data consists of multilingual tokens, enabling effective understanding and processing of multiple languages.

6. **Programming and Reasoning Skills**: Llama 3.1 excels in generating high-quality code and exhibits strong logical reasoning, problem-solving, and analytical capabilities.

7. 

In [14]:
for src in response.source_nodes:
    print("Node ID\t", src.node_id)
    print("Text\t", src.text)
    print("Score\t", src.score)
    print("-_" * 20)

Node ID	 a6ebe22b-5529-4e78-93a6-e7336d031600
Text	 GPT-2  GPT-3  and GPT-4  which were decoder-only architectures. Another well-known example is BERT (Bidirectional Encoder Representations from Transformers)  an encoder-only transformer mode used as a component in sentence embedding models.   Lets talk about BERT!BERT stands for Bidirectional Encoder Representations from Transformers. It is a language model by Google that uses a transformer architecture to understand and generate human-like language. BERT is designed to simultaneously process text in both directions  allowing it to capture context more effectively than traditional unidirectional models  which read text sequentially from left to right or right to left.   Example of Bidirectional CapabilityConsider the sentence:    The bank is situated on the _______ of the river.   In a unidirectional model  understanding the blank would primarily rely on the words before it  potentially leading to ambiguity about whether bank refers t

# Test gemini-1.5-flash Multi-Step


In [ ]:
from llama_index.core.indices.query.query_transform.base import StepDecomposeQueryTransform
from llama_index.core.query_engine.multistep_query_engine import MultiStepQueryEngine
from llama_index.llms.gemini import Gemini

llm_gemini = Gemini(model="models/gemini-1.5-flash")

step_decompose_transform = StepDecomposeQueryTransform(llm=llm_gemini, verbose=True)

query_engine_gemini = vector_index.as_query_engine(llm=llm_gemini)

query_engine_gemini = MultiStepQueryEngine(
    query_engine=query_engine_gemini,
    query_transform=step_decompose_transform,
    index_summary="Used to answer the Questions about RAG, Machine Learning, Deep Learning,LLMs and Generative AI",
)

In [ ]:
response_gemini = query_engine_gemini.query("Write about Llama 3.1 Model, BERT and PEFT")

> Current query: Write about Llama 3.1 Model, BERT and PEFT
> New query: What are Llama 3.1, BERT, and PEFT, and how do they relate to LLMs and generative AI?

> Current query: Write about Llama 3.1 Model, BERT and PEFT
> New query: What are the key characteristics and applications of Llama 3.1 and PEFT in the context of LLMs and generative AI?

> Current query: Write about Llama 3.1 Model, BERT and PEFT
> New query: What are the key characteristics and applications of BERT in the context of LLMs and generative AI?



In [ ]:
response_gemini.response

'Llama 3.1 is an open-source large language model known for its impressive performance, often matching or surpassing that of GPT-4 in various benchmarks while maintaining a lower inference cost. Its architecture allows for scalability, with versions ranging from smaller models that require fewer resources to a substantial 405 billion parameter version that demands significant GPU capabilities. The model can be deployed on-premises or through cloud services, making it versatile for different applications.\n\nPEFT, or Parameter-Efficient Fine-Tuning, is a library designed to enhance the fine-tuning process of large models like Llama 3.1. It employs techniques such as Low-Rank Adaptation (LoRA) to focus on a subset of parameters, allowing for efficient adjustments without the need for extensive computational resources. This method is particularly useful for customizing models for specific tasks, improving their performance while minimizing costs.\n\nBERT, on the other hand, is a discrimin

## Test Retriever on Multistep


In [ ]:
# import llama_index
# from llama_index.core.indices.query.schema import QueryBundle

# t = QueryBundle("How Retrieval Augmented Generation (RAG) work?")
# query_engine_gemini.retrieve(t)

## Subquestion Query Engine

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine

query_engine = vector_index.as_query_engine()

query_engine_tools = [
    QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name="LlamaIndex",
            description="Used to answer the Questions about RAG, Machine Learning, Deep Learning, and Generative AI. Note: Don't repeat the Same question",
        ),
    ),
]

sub_question_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    use_async=True,
)

response = sub_question_engine.query("Write about Llama 3.1 Model, BERT and PEFT")


Generated 5 sub questions.
[LlamaIndex] Q: What are the key features and improvements of the Llama 3.1 model compared to its predecessors?
[LlamaIndex] Q: How does BERT work and what are its main applications in natural language processing?
[LlamaIndex] Q: What is PEFT (Parameter-Efficient Fine-Tuning) and how does it enhance the performance of models like BERT?
[LlamaIndex] Q: What are the differences in architecture between Llama 3.1 and BERT?
[LlamaIndex] Q: In what scenarios is PEFT particularly beneficial for fine-tuning language models?
[LlamaIndex] A: The information provided does not include specific details about the architectural differences between Llama 3.1 and BERT. However, it highlights Llama 3.1's advancements, such as its large scale, extensive training on diverse data, and capabilities in reasoning and multilingual processing. For a comprehensive comparison, one would typically consider factors like model size, training methodology, context length, and specific archit

In [ ]:
response.response

'Llama 3.1 is a state-of-the-art model developed by Meta, notable for its extensive training on over 15 trillion tokens using more than 16,000 H100 GPUs. This model features an extended context length of 128K, enhancing its ability to process long text inputs. It demonstrates improved reasoning and coding capabilities, excels in multilingual processing, and supports zero-shot tool use, making it versatile for various applications. Llama 3.1 outperforms its predecessors in benchmark tests, particularly in mathematical and complex reasoning tasks.\n\nBERT, or Bidirectional Encoder Representations from Transformers, utilizes a transformer architecture that processes text bidirectionally, allowing it to capture the full context of sentences. It is pre-trained on tasks like Masked Language Modeling and Next Sentence Prediction, which help it generate contextualized word representations and understand sentence relationships. BERT is widely used in natural language processing applications suc

# HyDE Transform


In [ ]:
query_engine = vector_index.as_query_engine()

In [ ]:
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine.transform_query_engine import TransformQueryEngine

hyde = HyDEQueryTransform(include_original=True)
hyde_query_engine = TransformQueryEngine(query_engine, hyde)

In [ ]:
response = hyde_query_engine.query("Write about Llama 3.1 Model, BERT and PEFT")

In [ ]:
response.response

"Llama 3.1 405B is a significant advancement in the field of AI, developed by Meta. It stands out as the largest open-source model to date, trained on over 15 trillion tokens using more than 16,000 H100 GPUs. This extensive training has enabled it to achieve a 128K context length, which enhances its capabilities in reasoning, coding, and multilingual processing. The model excels in generating high-quality code and demonstrates strong logical reasoning and problem-solving skills. It has been shown to outperform proprietary models like GPT-4o and Claude 3.5 Sonnet in various benchmark tests, particularly in areas such as mathematical reasoning and multilingual support.\n\nBERT, or Bidirectional Encoder Representations from Transformers, is another influential model in the AI landscape, primarily designed for natural language understanding tasks. Unlike Llama 3.1, which focuses on a broader range of capabilities including coding and multilingual processing, BERT is specifically optimized 

In [ ]:
for src in response.source_nodes:
    print("Node ID\t", src.node_id)
    print("Text\t", src.text)
    print("Score\t", src.score)
    print("-_" * 20)

Node ID	 5624cdc8-2997-4e4d-82d1-c7383d389215
Text	 3.1 405B is Metas largest model  trained with over 15 trillion tokens. For this  Meta optimized the entire training stack and trained it on more than 16 000 H100 GPUs  making it the first Llama model trained at this scale.   According to Meta  this version of the original model (Llama 1 and Llama 2) has 128K context length  improved reasoning and coding capabilities. Meta has also upgraded both multilingual 8B and 70B models.   Key Features of Llama 3.1 40 5B:Llama 3.1 comes with a host of features and capabilities that appeal to The users  such as:   RAG & tool use  Meta states that you can use Llama system components to extend the model using zero-shot tool use and build agentic behaviors with RAG.   Multi-lingual  Llama 3 naturally supports multilingual processing. The pre-training data includes about 50% multilingual tokens and can process and understand multiple languages.   Programming and Reasoning  Llama 3 has powerful program

In [ ]:
query_bundle = hyde("Write about Llama 3.1 Model, BERT and PEFT")

In [ ]:
hyde_doc = query_bundle.embedding_strs[0]

In [ ]:
hyde_doc

"The Llama 3.1 model, developed by Meta, represents a significant advancement in the field of natural language processing (NLP). Building on the foundation laid by its predecessors, Llama 3.1 is designed to enhance the understanding and generation of human-like text. It incorporates a larger dataset and improved training techniques, which contribute to its ability to perform a wide range of language tasks, from text completion to question answering. The model's architecture is based on transformer technology, which allows it to capture complex patterns in language and generate coherent and contextually relevant responses.\n\nIn contrast, BERT (Bidirectional Encoder Representations from Transformers), developed by Google, is another influential model in the NLP landscape. BERT introduced the concept of bidirectional training, allowing the model to consider the context of a word based on all of its surrounding words, rather than just the words that precede it. This bidirectional approach